In [ ]:
### START FROM HERE
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# General installation of packages

!pip install cobra
import cobra
from cobra import Model, Reaction, Metabolite

# Import model
model=cobra.io.read_sbml_model('/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/Rt_IFO0880.xml') # Vittorio
#model=cobra.io.read_sbml_model('/content/drive/MyDrive/TalTech/Rt_IFO0880.xml') # Alina
model #to check it worked


#!pip install escher
#import escher
#from escher import Builder

In [ ]:
# Putative ethylene glycol pathway:

# ethylene glycol oxidase: ethylene glycol DH
# GRE2: RHTO_02276 - 12151 (LALDO3), RHTO_04342 - 11160 (LALDO3), RHTO_03406 - 10497 (LALDO3), RHTO_04716 - 12315 (), RHTO_00738 - 11786 ()
# ALD1/ALD6:
# YLL056C: RHTO_04070 - 9601 (), RHTO_04071 - 9602 ()

# glycolaldehyde oxidase: glycolaldehyde DH
# ALDH: RHTO_05835 - 12042 (GCALDD)
# ALD3
# ALD4
# ALD5

# glyoxylate reductase:
# GOR1: RHTO_05831 - 12051 (GLYCLTDxm)

In [ ]:
### Add EG as a metabolite

# 1. Add EG to the cytosol and the outside of the cell
model.add_metabolites([
            Metabolite(
            'eg_c',
            formula='C2H6O2',
            name='eg_c',
            compartment='c'
            ),
            Metabolite(
            'eg_e',
            formula='C2H6O2',
            name='eg_e',
            compartment='e'   
            ),
])


# 2. Create transport reaction to allow EG to enter the cell (from e to c)
eg_e=model.metabolites.get_by_id('eg_e')
eg_c=model.metabolites.get_by_id('eg_c')

EGt = Reaction('T_eg')
EGt.name = 'EG transport'
EGt.subsystem = 'EG Oxydation Pathway'
EGt.lower_bound = -999999
EGt.upper_bound = 999999

EGt.add_metabolites({
                eg_e: -1.0,
                eg_c: 1.0
})

model.add_reactions([EGt])
#model.reactions.get_by_id("T_eg") # check


# 3. Create exchange reaction to allow EG to enter the cell
EX_eg_e = Reaction('EX_eg_e')
EX_eg_e.name = 'EG exchange'
EX_eg_e.subsystem = 'EG Oxydation Pathway'
EX_eg_e.lower_bound = -999999
EX_eg_e.upper_bound = 999999

EX_eg_e.add_metabolites({
                eg_e: -1.0,
})

model.add_reactions([EX_eg_e])
#model.reactions.get_by_id("EX_eg_e") # check

# set bounds for EG exchange
#model.add_boundary(model.metabolites.get_by_id('eg_e'), type="exchange")

# check if it has been created (and the name); rxn are added at the bottom
#model.exchanges


In [ ]:
### Create the new reactions for EG consumption


## 1. EG oxidation to GAH (NAD+) - homologue to YLL056C (Sc)
eg_ox_nad = Reaction('EG_ox_nad')
eg_ox_nad.name = 'EG - NAD-dependent alcohol dehydrogenase'
eg_ox_nad.subsystem = 'EG Oxydation Pathway'
eg_ox_nad.lower_bound = -999999
eg_ox_nad.upper_bound = 999999

## 2. EG oxidation to GAH (NADP+) - homologue to GRE2 (Sc)
eg_ox_nadp = Reaction('EG_ox_nadp')
eg_ox_nadp.name = 'EG - NADP-dependent alcohol dehydrogenase'
eg_ox_nadp.subsystem = 'EG Oxydation Pathway'
eg_ox_nadp.lower_bound = -999999
eg_ox_nadp.upper_bound = 999999

## 3. Get the metabolites involved in the reactions
eg_c=model.metabolites.get_by_id("eg_c")        # get EG
nad_c=model.metabolites.get_by_id("nad_c")      # get NAD+
nadh_c=model.metabolites.get_by_id("nadh_c")    # get NADH
h_c=model.metabolites.get_by_id("h_c")          # get a proton
nadp_c=model.metabolites.get_by_id("nad_c")     # get NADP+
nadph_c=model.metabolites.get_by_id("nadh_c")   # get NADPH
gah_c=model.metabolites.get_by_id('gcald_c')    # get GAH

## 4. Add metabolites to reactions and change bounds

# 4a. NAD+ dependent
eg_ox_nad.add_metabolites({
              eg_c: -1.0,
              nad_c: -1.0,
              gah_c: 1.0,
              nadh_c: 1.0,
              h_c: 1.0,
})

#eg_ox_nad.reaction  # to check

# change bounds
eg_ox_nad.lower_bound = 0



# 4b. NADP+ dependent
eg_ox_nadp.add_metabolites({
              eg_c: -1.0,
              nadp_c: -1.0,
              gah_c: 1.0,
              nadph_c: 1.0,
              h_c: 1.0,
})

#eg_ox_nadp.reaction  # to check

# change bounds
eg_ox_nadp.lower_bound = 0


## 5. Add reactions to model
model.add_reactions([eg_ox_nad])
model.add_reactions([eg_ox_nadp])


In [ ]:
### Change bounds for the reactions GA <-> GOX

# 1. Glyoxylate reductase (NAD), mitochondrial
ga_ox_mit=model.reactions.get_by_id('GLYCLTDxm')
ga_ox_mit.bounds = (-1000, 1000)
#ga_ox_mit # to check

# 2. Glyoxylate reductase (NADP), cytosol
ga_ox_c=model.reactions.get_by_id('GLYCLTDy')
ga_ox_c.bounds = (-1000, 1000)
#ga_ox_c # to check



### Allow glycolate accumulation
# constraint glycolate
EX_glyclt_e=model.reactions.get_by_id('EX_glyclt_e')
EX_glyclt_e.upper_bound=0.1
EX_glyclt_e.lower_bound=0.1
EX_glyclt_e

In [ ]:
# Save model:
# json: escher maps
# xml
# mat

### CHECK THE NAME OF THE FILES EVERY TIME!!! ###

cobra.io.json.save_json_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/EG02_Glc_results/Rt_IFO0880_EG_NADP.json')
cobra.io.save_matlab_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/EG02_Glc_results/Rt_IFO0880_EG_NADP.mat')
cobra.io.write_sbml_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/EG02_Glc_results/Rt_IFO0880_EG_NADP.xml')
#cobra.io.json.save_json_model(model,'/content/drive/MyDrive/TalTech/Rt_IFO0880_EG.json') # Alina
#cobra.io.save_matlab_model(model,'/content/drive/MyDrive/TalTech/Rt_IFO0880_EG.mat') # Alina
#cobra.io.write_sbml_model(model,'/content/drive/MyDrive/TalTech/Rt_IFO0880_EG.xml') # Alina


In [ ]:
# Simulations:

# check objective
print(model.objective.expression)
print(model.objective.direction)

# if necessary to change the objective:
#model.objective = "ATPM" # or any other reaction

In [ ]:
### Setting the parameters for the simulation (e.g. medium)

# set glucose uptake to 0
medium = model.medium
medium["EX_glc__D_e"] = 0.0
model.medium = medium

# set EG uptake to 1
medium = model.medium
medium["EX_eg_e"] = 1.0
model.medium = medium

model.medium    # check it worked

In [ ]:
# Run flux balance analysis, FBA (optimization function):

solution = model.optimize()
solution.objective_value
#print(solution)

In [ ]:
# Run alternative FBA:
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution

In [ ]:
solution.fluxes

In [ ]:
cobra.flux_analysis

In [ ]:
# Export data (for Excher):

import pandas as pd
data = solution.fluxes
#data = pfba_solution.fluxes
data.to_csv('/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/EG02_Glc_results/Rt_IFO0880_EG_and_Glc._fluxes.csv')
#data.to_csv('/content/drive/MyDrive/TalTech/Rt_IFO0880_EG__NADP.csv') # Alina

In [ ]:
# Other code:

model.metabolites.nadh_x.summary()
# model.summary()

In [ ]:
model.metabolites.nadh_m.summary()

In [ ]:
model.metabolites.nadh_c.summary()

In [ ]:
model.metabolites.nadph_x.summary()


In [ ]:
model.metabolites.nadph_m.summary()


In [ ]:
model.metabolites.nadph_c.summary()
